In [27]:
import tensorflow as tf
import core.utils as utils
from core.config import cfg
from core.yolov4 import filter_boxes
from tensorflow.python.saved_model import tag_constants
from PIL import Image
import cv2
import numpy as np
from core.yolov4 import YOLO, decode, compute_loss, decode_train

In [28]:
flg_tiny = False
flg_model = 'yolov4'

In [30]:
INPUT_SIZE = 416
STRIDES, ANCHORS, NUM_CLASS, XYSCALE = utils.load_config(flg_tiny , flg_model)
CLASSES = utils.read_class_names(cfg.YOLO.CLASSES)

In [4]:
image_path = r'C:\yolo\datasets\VOC2007\JPEGImages\000022.jpg'

In [5]:
original_image = cv2.imread(image_path)
original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
image_data = cv2.resize(original_image, (input_size, input_size))
image_data = image_data / 255.

In [6]:
input_layer = tf.keras.layers.Input([input_size, input_size, 3])

In [7]:
freeze_layers = utils.load_freeze_layer(flg_model, flg_tiny)
feature_maps = YOLO(input_layer, NUM_CLASS, flg_model, flg_tiny)
if flg_tiny:
    bbox_tensors = []
    for i, fm in enumerate(feature_maps):
        if i == 0:
            bbox_tensor = decode_train(fm, cfg.TRAIN.INPUT_SIZE // 16, NUM_CLASS, STRIDES, ANCHORS, i, XYSCALE)
        else:
            bbox_tensor = decode_train(fm, cfg.TRAIN.INPUT_SIZE // 32, NUM_CLASS, STRIDES, ANCHORS, i, XYSCALE)
        bbox_tensors.append(fm)
        bbox_tensors.append(bbox_tensor)
else:
    bbox_tensors = []
    for i, fm in enumerate(feature_maps):
        if i == 0:
            bbox_tensor = decode_train(fm, cfg.TRAIN.INPUT_SIZE // 8, NUM_CLASS, STRIDES, ANCHORS, i, XYSCALE)
        elif i == 1:
            bbox_tensor = decode_train(fm, cfg.TRAIN.INPUT_SIZE // 16, NUM_CLASS, STRIDES, ANCHORS, i, XYSCALE)
        else:
            bbox_tensor = decode_train(fm, cfg.TRAIN.INPUT_SIZE // 32, NUM_CLASS, STRIDES, ANCHORS, i, XYSCALE)
        bbox_tensors.append(fm)
        bbox_tensors.append(bbox_tensor)

In [8]:
model = tf.keras.Model(input_layer, bbox_tensors)

In [9]:
model.load_weights("./checkpoints/yolov4")

In [26]:
pred_bbox  = model(np.expand_dims(image_data,0))
# pred_bbox = [tf.reshape(x, (tf.shape(x)[0], -1, tf.shape(x)[-1])) for x in pred_bbox]
print(pred_bbox)

[<tf.Tensor: shape=(1, 52, 52, 255), dtype=float32, numpy=
array([[[[ 6.74825253e-20,  3.73780220e-21,  4.44785646e-20, ...,
           5.86060566e-20, -4.90703573e-21, -3.51307871e-20],
         [ 8.00989231e-20,  3.40454066e-20, -1.07186145e-20, ...,
           2.79414987e-20,  9.35032686e-21,  4.41615735e-20],
         [ 1.55613211e-19,  1.62606544e-19, -7.18960767e-20, ...,
          -2.26448415e-20,  1.46322337e-20, -6.07801134e-20],
         ...,
         [ 1.97361081e-19,  3.19540181e-19, -1.43476881e-19, ...,
          -4.96662892e-20, -1.90013177e-20,  4.73577315e-20],
         [ 2.05555817e-19,  4.03506664e-19, -9.02079749e-20, ...,
          -4.84868330e-20,  9.35983055e-21,  1.15999906e-19],
         [ 1.62559175e-19,  2.02266494e-19, -1.08385269e-19, ...,
          -6.04111392e-20,  4.72496778e-20, -1.15099315e-21]],

        [[ 8.76508430e-20, -2.32068768e-20,  7.64128189e-20, ...,
           3.75009245e-20, -4.30922067e-20, -6.39520150e-20],
         [ 7.24887257e-20,  2

In [22]:
for key, value in pred_bbox.items():
    boxes = value[:, :, 0:4]
    pred_conf = value[:, :, 4:]

AttributeError: 'list' object has no attribute 'items'